Данный проект был выполнен в рамках обучения МФТИ.  

От заказчика поступил размеченный датасет с отзывами о мобильных телефонах и тональности отзывов (положительный-отрицательный). Датасет маленький и содержит всего 100 строк. На таком датасете не возможно будет составить обучающую и тестовую выборку. Так же у заказчика требование - чтобы на тестовой выборке качество было не менее 80%.  
  
Выход в сложившейся ситуации - парсить отзывы с сайта такой же тематики.  

Я нашёл сайт slonrekomenduet.com. На нём есть много отзывов о мобильных телефонах и смартфонах.  

На сайте есть как отзывы, так и баллы. Скорее всего 4-5 баллов положительный отзыв, остальное - отрицательный.  
Попробую пойти по следющему сценарию:

1. Спарсить только отзывы всех телефонов и смартфонов.
2. Обучить модель по вариану 1-3 балла neg, 4-5 pos.

In [2]:
import pandas as pd
import requests
import bs4

Баллы закодированы в звёздах, поэтому нужно будет считать активные звёзды как балл.  

In [3]:
#парсинг страницы производителей
def get_prods(url):
    #получить все теги 'a' списка 'ul'
    req = requests.get(url)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    parser = parser.find('ul')
    prod_urls = parser.findAll('a', href=True)
    #добавить их в массив
    prods=[]
    for prod in prod_urls:
        prods.append('https://slonrekomenduet.com'+prod['href'])
        
    return prods

In [4]:
#парсинг страницы моделей
def get_models(url):
    #найти на странице paginator
    req = requests.get(url)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    parser = parser.find('div', attrs={'id':'paginator'})
    page_urls = parser.findAll('a', href=True)
    #если есть url других страниц, то добавить их в массив url
    pages=[url]
    if len(page_urls)>0:
        for page in page_urls:
            pages.append('https://slonrekomenduet.com'+page['href'])
    
    model_urls=[]
    for page_model in pages:
        req = requests.get(page_model)
        parser = bs4.BeautifulSoup(req.text, 'lxml')
        parser = parser.find('div', attrs={'id':'models'})
        containers = parser.findAll('div', attrs={'class':'model'})
        #добавить их в массив
        for con in containers:
            model=con.find('a', href=True)
            model_urls.append('https://slonrekomenduet.com'+model['href'])
    return model_urls

In [5]:
#получить баллы
def get_stars(stars):
    stars_count=[]
    for x in stars:
        stars_count.append(str(x).count('br-active'))
    return stars_count

#получить текст
def get_text(text):
    rev_text=[]
    for x in text:
        text=x.text
        text=text.replace('Достоинства','Достоинства ')
        text=text.replace('Недостатки','Недостатки ')
        text=text.replace('Комментарий','Комментарий ')
        rev_text.append(text)
    return rev_text

In [6]:
#парсинг страницы отзывов
def get_rews(url):
    #получить страницу отзывов, найти на ней пагинацию и спарсить все сраницы отзывов модели
    #найти на странице paginator
    req = requests.get(url)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    parser = parser.find('div', attrs={'id':'paginator'})
    #если есть url других страниц, то добавить их в массив url
    pages=[url]
    if parser:
        page_urls = parser.findAll('a', href=True)
        for page in page_urls:
            pages.append('https://slonrekomenduet.com'+page['href'])
    
    rev_stars=[]
    rev_text=[]
    for page_rew in pages:
        req = requests.get(page_rew)
        parser = bs4.BeautifulSoup(req.text, 'lxml')
        parser = parser.find('div', attrs={'id':'user_reviews'})
        review_meta = parser.findAll('div', attrs={'class':'br-theme-css-stars'})
        review_text = parser.findAll('div', attrs={'class':'comment_text'})
        stars=get_stars(review_meta)
        text=get_text(review_text)
        if len(stars)==len(text):
            rev_stars.extend(stars)
            rev_text.extend(text)
    return rev_stars,rev_text

In [7]:
#массив баллов
rev_stars=[]
#массив отзывов
rev_text=[]
#массив страниц разделов
pages_cat=[
    'https://slonrekomenduet.com/category/phones.html',
    'https://slonrekomenduet.com/category/smartphones.html'
]
#массив страниц производителей
pages_prods=[]
#массив страниц моделей
pages_models=[]

#получаю всех производителей
for page_cat in pages_cat:
    pages_prods.extend(get_prods(page_cat))
    
print('get prods',len(pages_prods))

#получаю все модели
for page_prod in pages_prods:
    pages_models.extend(get_models(page_prod))
    
print('get models',len(pages_models))

#получаю все отзывы
for page_model in pages_models:
    stars,text=get_rews(page_model)
    rev_stars.extend(stars)
    rev_text.extend(text)

print('get rews fiished:',len(rev_text))

get prods 112
get models 2185
get rews fiished: 187077


In [8]:
len(rev_stars)

187077

In [9]:
len(rev_text)

187077

Я получил 187077 отзывов о телефонах.

In [10]:
d = {'text': rev_text, 'stars': rev_stars}
df = pd.DataFrame(data=d)

In [11]:
df.head()

,text,stars
0,На белой коробке-упаковочке смешное фото: то-л...,1
1,Хороший телефон .Соотношение цены и качества-о...,5
2,Достоинства : \rподдержка карты памяти до 32 G...,4
3,"Упаковка от телефона не очень понравилась, а и...",5
4,Достоинства : Первый день пользования. Ощущени...,4


Необходимо удалить символ перевода строки _"\r"_, а так же перевести баллы в показатель 0 и 1.

In [12]:
df.text=df.text.str.replace('\r', ' ', regex=True)
df.loc[df.stars>3,'class']=1
df.loc[df.stars<=3,'class']=0

In [13]:
df.head()

,text,stars,class
0,На белой коробке-упаковочке смешное фото: то-л...,1,0.0
1,Хороший телефон .Соотношение цены и качества-о...,5,1.0
2,Достоинства : поддержка карты памяти до 32 Gb...,4,1.0
3,"Упаковка от телефона не очень понравилась, а и...",5,1.0
4,Достоинства : Первый день пользования. Ощущени...,4,1.0


In [14]:
df['class'].value_counts()

1.0    142441
0.0     44636
Name: class, dtype: int64

Составлю датасет для обучения.

In [4]:
X=df.text
y=df['class']

Сохраню данные на всякий случай.

In [15]:
df.to_csv('rew_data.csv',index=False)

In [3]:
df=pd.read_csv('rew_data.csv')

Теперь можно попробовать различные модели

In [5]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np
import nltk
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

Я хочу попробовать LinearSVC и LogisticRegression, так как по моему опыту они наиболее эффективны при работе с текстом.  
Параметры одинаковые: n-gramm = 2 и лемматизатор pymystem3 с русскоязычной поддержкой. Векторизатор текста я использую TfidfVectorizer.

In [6]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from sklearn.preprocessing import FunctionTransformer

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text(X):
    prep_text=[]
    for text in X:
        tokens = mystem.lemmatize(text.lower())
        tokens = [token for token in tokens if token not in russian_stopwords\
                  and token != " " \
                  and token.strip() not in punctuation]
    
        text = " ".join(tokens)
    
        prep_text.append(text)
    return prep_text

[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
%%time

tfdf_svc_clf = Pipeline([('norm',FunctionTransformer(preprocess_text,validate = False)),
                    ('vect', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf', LinearSVC())])

tfdf_log_clf = Pipeline([('norm',FunctionTransformer(preprocess_text,validate = False)),
                        ('vect', TfidfVectorizer(ngram_range=(1, 2))),
                         ('clf', LogisticRegression())])

tfdf_svc_res=cross_val_score(tfdf_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=2)

print("tfdf_svc mean:",np.mean(tfdf_svc_res))
print("tfdf_svc std:",np.std(tfdf_svc_res))

tfdf_log_res=cross_val_score(tfdf_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=2)

print("tfdf_log mean:",np.mean(tfdf_log_res))
print("tfdf_log std:",np.std(tfdf_log_res))

tfdf_svc mean: 0.896438519537301
tfdf_svc std: 0.007574252773938955
tfdf_log mean: 0.8920926266556084
tfdf_log std: 0.004048556700167318
CPU times: user 10.6 s, sys: 7.1 s, total: 17.7 s
Wall time: 2h 27min 23s


Неплохо, теперь обучу модель на полных данных

In [9]:
tfdf_log_clf.fit(X,y)

Pipeline(memory=None,
     steps=[('norm', FunctionTransformer(accept_sparse=False,
          func=<function preprocess_text at 0x7f1ab32f4b70>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='st...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [10]:
tfdf_log_clf.score(X,y)

0.9316057024647605

In [11]:
tfdf_svc_clf.fit(X,y)

Pipeline(memory=None,
     steps=[('norm', FunctionTransformer(accept_sparse=False,
          func=<function preprocess_text at 0x7f1ab32f4b70>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='st...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [12]:
tfdf_svc_clf.score(X,y)

0.997893915339673

Скорее всего я выберу SVC  
Теперь прочитаю данные из файла и сделаю по ним прогноз.

In [13]:
import codecs
fileObj = codecs.open("test.csv", "r", "utf_8_sig" )
parser = bs4.BeautifulSoup(fileObj.read(), 'html.parser')
review_meta = parser.findAll('review')

In [14]:
test_text=[]
for x in review_meta:
    test_text.append(x.text.replace('\n',''))

In [15]:
res=tfdf_svc_clf.predict(test_text)

In [16]:
res

array([0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.])

In [17]:
sub=pd.read_csv('sample_submission.csv')

In [18]:
sub.y=res

In [19]:
sub.loc[sub.y==0,'y']='neg'
sub.loc[sub.y==1,'y']='pos'

In [20]:
sub.to_csv('result_svc.csv',index=False)

In [21]:
res=tfdf_log_clf.predict(test_text)
sub.y=res
sub.loc[sub.y==0,'y']='neg'
sub.loc[sub.y==1,'y']='pos'
sub.to_csv('result_log.csv',index=False)

На kaggle https://www.kaggle.com/c/product-reviews-sentiment-analysis/submissions получилась точность валидации для svc 98%, для логистической регресии 96%, в итоге я превзошёл требования заказчика по точности классификации на 18%.